In [1]:
import reskit

In [1]:
import geokit as gk
import pandas as pd
import numpy as np
import pvlib
from scipy.interpolate import RectBivariateSpline

import reskit as rk
from reskit import solarpower
from reskit.workflows.solar import SolarWorkflowGenerator

In [2]:
# placements['capacity'] = 2500
# placements['tilt'] = placements['lat']*0.76
# placements['azimuth'] = 180
# placements['elev'] = 300

# placements.to_csv('c:\\users\\s.ryberg\\fzj\\repos\\res\\reskit\\test\\data\\module_placements.csv', index=False)

In [3]:
placements = pd.read_csv(rk._TEST_DATA_['module_placements.csv'])
merra_path=rk._TEST_DATA_['weather_data']

In [4]:
wf = SolarWorkflowGenerator(placements)

# tests
assert isinstance( wf.locs, gk.LocationSet)
assert wf.locs.count == placements.shape[0]

In [5]:
wf.read(
    variables=['surface_wind_speed', "surface_pressure", "surface_air_temperature", "surface_dew_temperature", "global_horizontal_irradiance"],
    source_type="MERRA",
    path=merra_path,
    set_time_index=True,
    verbose=False
)

# tests
assert "surface_wind_speed" in wf.sim_data
assert "surface_pressure" in wf.sim_data
assert "surface_air_temperature" in wf.sim_data
assert "global_horizontal_irradiance" in wf.sim_data

assert wf.sim_data['global_horizontal_irradiance'].shape[0] == len(wf.time_index)
assert wf.sim_data['global_horizontal_irradiance'].shape[1] == wf.locs.count

assert wf.surface_wind_speed_height == 2
# print(wf.sim_data['global_horizontal_irradiance'].mean(), wf.sim_data['global_horizontal_irradiance'].std())
assert np.isclose( wf.sim_data['global_horizontal_irradiance'].mean(), 29.051229857488853 )
assert np.isclose( wf.sim_data['global_horizontal_irradiance'].std(), 57.41876854447587)

In [6]:
wf.determine_solar_position()

# tests 
assert "solar_azimuth" in wf.sim_data
assert "apparent_solar_zenith" in wf.sim_data
assert "apparent_solar_elevation" in wf.sim_data

# print(wf.sim_data['solar_zenith'].mean(), wf.sim_data['solar_zenith'].std())
assert np.isclose( wf.sim_data['apparent_solar_zenith'].mean(), 109.2598193185083 )
assert np.isclose( wf.sim_data['apparent_solar_zenith'].std(), 26.62641165426258)


In [7]:
wf.filter_positive_solar_elevation()

# tests 
assert wf._time_sel_.sum() == 27
assert wf._sim_shape_ == (27,560)
assert wf.sim_data['global_horizontal_irradiance'].shape == wf._sim_shape_

In [8]:
wf.determine_extra_terrestrial_irradiance(model="spencer", solar_constant=1370)

# print(wf.sim_data['extra_terrestrial_irradiance'].mean(), wf.sim_data['extra_terrestrial_irradiance'].std())
assert np.isclose( wf.sim_data['extra_terrestrial_irradiance'].mean(), 1418.0397631172639   )
assert np.isclose( wf.sim_data['extra_terrestrial_irradiance'].std(), 0.015717104724686137)

In [9]:
wf.determine_air_mass(model='kastenyoung1989')
    
# print(wf.sim_data['air_mass'].mean(), wf.sim_data['air_mass'].std())
assert np.isclose( wf.sim_data['air_mass'].mean(), 10.355618714207562 )
assert np.isclose( wf.sim_data['air_mass'].std(), 9.517938562733685)

In [10]:
wf.apply_DIRINT_model()

# print(wf.sim_data['direct_normal_irradiance'].mean(), wf.sim_data['direct_normal_irradiance'].std())
assert np.isclose( wf.sim_data['direct_normal_irradiance'].mean(), 108.46491710266537  )
assert np.isclose( wf.sim_data['direct_normal_irradiance'].std(), 174.48104062756954)

In [11]:
wf.diffuse_horizontal_irradiance_from_trigonometry()

# print(wf.sim_data['diffuse_horizontal_irradiance'].mean(), wf.sim_data['diffuse_horizontal_irradiance'].std())
assert np.isclose( wf.sim_data['diffuse_horizontal_irradiance'].mean(), 51.417301002067795   )
assert np.isclose( wf.sim_data['diffuse_horizontal_irradiance'].std(), 39.27241552930912)

In [12]:
wf.permit_single_axis_tracking(max_angle=90, backtrack=True, gcr=2.0/7.0)

# print(wf.sim_data['system_tilt'].mean(), wf.sim_data['system_tilt'].std())
assert np.isclose( wf.sim_data['system_tilt'].mean(), 49.672198525949106  )
assert np.isclose( wf.sim_data['system_tilt'].std(), 10.248233044342607)

C:\Users\s.ryberg\AppData\Local\Continuum\anaconda3\envs\sevpy\lib\site-packages\pandas\util\__init__.py:12: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing


In [13]:
wf.determine_angle_of_incidence()

# print(wf.sim_data['angle_of_incidence'].mean(), wf.sim_data['angle_of_incidence'].std())
assert np.isclose( wf.sim_data['angle_of_incidence'].mean(), 38.981053931482315 )
assert np.isclose( wf.sim_data['angle_of_incidence'].std(), 17.920415717356125)

In [14]:
wf.estimate_plane_of_array_irradiances(transposition_model="perez")

# print(wf.sim_data['poa_global'].mean(), wf.sim_data['poa_global'].std())
assert np.isclose( wf.sim_data['poa_global'].mean(), 162.60480586176368  )
assert np.isclose( wf.sim_data['poa_global'].std(), 191.10117326401752)

In [15]:
wf.cell_temperature_from_sandia_method()

# print(wf.sim_data['cell_temperature'].mean(), wf.sim_data['cell_temperature'].std())
assert np.isclose( wf.sim_data['cell_temperature'].mean(), 6.311121740017978   )
assert np.isclose( wf.sim_data['cell_temperature'].std(), 5.943765110179128)

In [16]:
wf.apply_angle_of_incidence_losses_to_poa()

# print(wf.sim_data['poa_global'].mean(), wf.sim_data['poa_global'].std())
assert np.isclose( wf.sim_data['poa_global'].mean(), 159.22388470705638     )
assert np.isclose( wf.sim_data['poa_global'].std(), 188.41685577086287)

In [17]:
wf.simulate_with_interpolated_single_diode_approximation()

# print(wf.sim_data['capacity_factor'].mean(), wf.sim_data['capacity_factor'].std())
assert np.isclose( wf.sim_data['capacity_factor'].mean(), 0.215093351772682  )
assert np.isclose( wf.sim_data['capacity_factor'].std(), 0.2532449604453645)

# print(wf.sim_data['total_system_generation'].mean(), wf.sim_data['total_system_generation'].std())
assert np.isclose( wf.sim_data['total_system_generation'].mean(), 537.733379431705      )
assert np.isclose( wf.sim_data['total_system_generation'].std(), 633.1124011134112)

C:\Users\s.ryberg\AppData\Local\Continuum\anaconda3\envs\sevpy\lib\site-packages\pvlib\pvsystem.py:1918: RuntimeWarning: overflow encountered in exp
  np.exp((-I[idx_p] + IL[idx_p] + I0[idx_p]) /


In [18]:
if 'capacity' in wf.placements.columns:
    del wf.placements['capacity']
    wf.placements['modules_per_string'] = 2
    wf.placements['strings_per_inverter'] = 1
    wf.placements['number_of_inverters'] = 5

wf.apply_inverter_losses(inverter="SolarBridge_Technologies__P250HV_208_240_xxx_208V__CEC_2013_")

# print(wf.sim_data['capacity_factor'].mean(), wf.sim_data['capacity_factor'].std())
assert np.isclose( wf.sim_data['capacity_factor'].mean(), 0.17707640078814754    )
assert np.isclose( wf.sim_data['capacity_factor'].std(), 0.18471246597892066)

# print(wf.sim_data['total_system_generation'].mean(), wf.sim_data['total_system_generation'].std())
assert np.isclose( wf.sim_data['total_system_generation'].mean(), 425.6775013826436    )
assert np.isclose( wf.sim_data['total_system_generation'].std(), 444.03399121604696)

In [19]:
wf.apply_loss_factor(0.20, variables=['capacity_factor','total_system_generation'])

# print(wf.sim_data['capacity_factor'].mean(), wf.sim_data['capacity_factor'].std())
assert np.isclose( wf.sim_data['capacity_factor'].mean(), 0.14166112063051803  )
assert np.isclose( wf.sim_data['capacity_factor'].std(), 0.14776997278313653)

# print(wf.sim_data['total_system_generation'].mean(), wf.sim_data['total_system_generation'].std())
assert np.isclose( wf.sim_data['total_system_generation'].mean(), 340.54200110611487 )
assert np.isclose( wf.sim_data['total_system_generation'].std(), 355.22719297283754)

In [17]:
## TEST THESIS WORKFLOW
import pandas as pd
import reskit as rk
import numpy as np
from reskit.workflows.solar import openfield_pv_with_MERRA_Ryberg2019

xds = openfield_pv_with_MERRA_Ryberg2019(
    placements = pd.read_csv(rk._TEST_DATA_['module_placements.csv']),
    merra_path = rk._TEST_DATA_['weather_data'],
    global_solar_atlas_ghi_path = rk._TEST_DATA_['gsa-ghi-like.tif'],
)


# print(xds['capacity_factor'].fillna(0).mean(), xds['capacity_factor'].fillna(0).std())
assert np.isclose( xds['capacity_factor'].fillna(0).mean(), 0.04377396 )
assert np.isclose( xds['capacity_factor'].fillna(0).std(), 0.10127316)

In [21]:
## TEST SARAH WORKFLOW
import pandas as pd
import reskit as rk
import numpy as np
from reskit.workflows.solar import openfield_pv_with_SARAH_Ryberg2020

xds = openfield_pv_with_SARAH_Ryberg2020(
    placements = pd.read_csv(rk._TEST_DATA_['module_placements.csv']),
    sarah_path = rk._TEST_DATA_['sarah-like'],
    era5_path = rk._TEST_DATA_['era5-like']
)


# print(xds['capacity_factor'].fillna(0).mean())
# print(xds['capacity_factor'].fillna(0).std())
assert np.isclose( xds['capacity_factor'].fillna(0).mean(), 0.14953889 )
assert np.isclose( xds['capacity_factor'].fillna(0).std(), 0.25260633)

C:\Users\s.ryberg\AppData\Local\Continuum\anaconda3\envs\sevpy\lib\site-packages\pvlib\irradiance.py:991: RuntimeWarning: invalid value encountered in true_divide
  eps = ((dhi + dni) / dhi + kappa * (z ** 3)) / (1 + kappa * (z ** 3))
C:\Users\s.ryberg\AppData\Local\Continuum\anaconda3\envs\sevpy\lib\site-packages\pvlib\irradiance.py:1002: RuntimeWarning: invalid value encountered in less
  ebin[eps < 1.065] = 1
C:\Users\s.ryberg\AppData\Local\Continuum\anaconda3\envs\sevpy\lib\site-packages\pvlib\irradiance.py:1003: RuntimeWarning: invalid value encountered in greater_equal
  ebin[(eps >= 1.065) & (eps < 1.23)] = 2
C:\Users\s.ryberg\AppData\Local\Continuum\anaconda3\envs\sevpy\lib\site-packages\pvlib\irradiance.py:1003: RuntimeWarning: invalid value encountered in less
  ebin[(eps >= 1.065) & (eps < 1.23)] = 2
C:\Users\s.ryberg\AppData\Local\Continuum\anaconda3\envs\sevpy\lib\site-packages\pvlib\irradiance.py:1004: RuntimeWarning: invalid value encountered in greater_equal
  ebin[(eps